In [1]:
import os
import glob
import rasterio as rio
from rasterio.mask import raster_geometry_mask
from rasterio import plot
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import pyproj
from shapely.geometry import shape, MultiPolygon
from shapely.ops import transform

In [5]:
# Functions

# Get panel masks

def get_panel_masks(geojson_path, ref_img_path):
    '''This function generates a mask that leaves only pixels inside 
    vine panels unmasked. 
    
    Inputs:
    geojson_path: path to a geojson file with the panel geometries. 
    Must be in same CRS as image, otherwise un-comment code section starting with "dst_crs".
    ref_img_path: path to the .tif file for the image
    
    Outputs:
    A set of masks, as Boolean arrays. True values are masked pixels.
    False values are unmasked pixels.
    '''
    with open(geojson_path, 'r') as f:
        panels = json.load(f)
        features = panels['features']
        features_ordered = sorted(features, key= lambda feature:(feature['properties']['row'],feature['properties']['panel'])) # ground truth order.
        panel_aois = [f['geometry'] for f in features_ordered]
    
    with rio.open(ref_img_path) as src:
        dst_crs = src.crs
        
        # geojson features (the field block geometries)
        # are often in WGS84
        # project these to the image coordinates
        wgs84 = pyproj.CRS('EPSG:4326')

        project = pyproj.Transformer.from_crs(wgs84, dst_crs, always_xy=True).transform
        proj_panel_aois = [transform(project, shape(p)) for p in panel_aois]

        masks = [raster_geometry_mask(src, [p], crop=False)[0]
                 for p in proj_panel_aois]
       
    return masks
   

# Get reflectance df

def get_rfl(img_path, masks):
    '''
    Function to extract per-panel reflectance values for 4-band SkySat images.
    Calculates the mean reflectance values for the pixels in each panel.
    The output is formatted as a pandas dataframe.
    
    Inputs:
    img_path (str) - abs. path to the image       
    masks (list) - a list of masks that correspond to the panel aois. Each mask item is a Boolean array. 
    '''
    
    with rio.open(img_path,'r') as img:
        def _get_stats(mask, panel_number):
            panel_blue = np.ma.mean(np.ma.array(img.read(1), mask=mask))/10000
            panel_green = np.ma.mean(np.ma.array(img.read(2), mask=mask))/10000
            panel_red = np.ma.mean(np.ma.array(img.read(3), mask=mask))/10000
            panel_nir = np.ma.mean(np.ma.array(img.read(4), mask=mask))/10000
            
            return {'acquired': img_path.split('/')[-1].split('_')[-5], # date from image filename
                    'plot': panel_number + 1,    #add one - panels are numbered starting at 1 in the 'real world'
                    'blue': panel_blue,
                    'green': panel_green,
                    'red': panel_red,
                    'nir': panel_nir}
        data = [_get_stats(m, i) for i, m in enumerate(masks)]  # sets the parameters for the nested function as the mask (m) and corresponding panel number (i). 
        df = pd.DataFrame(data)
    
    return df

In [3]:
# image directory containing co-registered SkySat scenes 

img_dir = '/Users/kathleenkanaley/Desktop/grapes_in_space/images_work/images_analysis_ready/2020/SkySat/'

# image pathnames
img_paths = glob.glob(img_dir+'2020*.tif')

# geojson with panel geometries

json_path = '/Users/kathleenkanaley/Desktop/grapes_in_space/map/qgis_layers/crit_panels_geojson.geojson'
json_path
img_paths

['/Users/kathleenkanaley/Desktop/grapes_in_space/images_work/images_analysis_ready/2020/SkySat/20200710_155028_ssc3d2_0012_ar.tif',
 '/Users/kathleenkanaley/Desktop/grapes_in_space/images_work/images_analysis_ready/2020/SkySat/20200812_153924_ssc3d2_0013_ar.tif',
 '/Users/kathleenkanaley/Desktop/grapes_in_space/images_work/images_analysis_ready/2020/SkySat/20200625_154704_ssc12d2_0013_ar.tif']

In [6]:
# Get dataframes for all images in directory

master_df = pd.DataFrame()

for i in img_paths:
    masks = get_panel_masks(json_path, i)
    df = get_rfl(i, masks)
    master_df = pd.concat([master_df, df], axis=0)
    master_df = master_df.sort_values(by=['acquired', 'plot'])

master_df

,acquired,plot,blue,green,red,nir
0,20200625,1,0.051447,0.072572,0.061172,0.303591
1,20200625,2,0.055840,0.074537,0.063650,0.284137
2,20200625,3,0.051873,0.070787,0.058603,0.277567
3,20200625,4,0.052025,0.073668,0.061257,0.285911
4,20200625,5,0.047086,0.067566,0.049576,0.312017
...,...,...,...,...,...,...
315,20200812,316,0.030579,0.057362,0.035652,0.375034
316,20200812,317,0.030440,0.059553,0.035480,0.392890
317,20200812,318,0.030557,0.058318,0.032775,0.391243
318,20200812,319,0.031516,0.059703,0.033959,0.410013


In [7]:
# Append corresponding row and panel numbers for each plot 

panel_csv = pd.read_csv('/Users/kathleenkanaley/Desktop/grapes_in_space/map/qgis_layers/crit_panels.csv')
cols = panel_csv.iloc[:,1:] # adjust this according to the .csv
#cols
appended = pd.concat([master_df, cols],axis=1)
appended

,acquired,plot,blue,green,red,nir,row,panel
0,20200625,1,0.051447,0.072572,0.061172,0.303591,1,1
1,20200625,2,0.055840,0.074537,0.063650,0.284137,1,2
2,20200625,3,0.051873,0.070787,0.058603,0.277567,1,3
3,20200625,4,0.052025,0.073668,0.061257,0.285911,1,4
4,20200625,5,0.047086,0.067566,0.049576,0.312017,1,5
...,...,...,...,...,...,...,...,...
315,20200812,316,0.030579,0.057362,0.035652,0.375034,20,12
316,20200812,317,0.030440,0.059553,0.035480,0.392890,20,13
317,20200812,318,0.030557,0.058318,0.032775,0.391243,20,14
318,20200812,319,0.031516,0.059703,0.033959,0.410013,20,15


In [8]:
appended.acquired.unique()


array(['20200625', '20200710', '20200812'], dtype=object)

In [13]:
scout_csv = pd.read_csv('scout_2020_gdf.csv')
print(scout_csv.Date.unique())
print(scout_csv.columns)

['2020-06-18' '2020-06-25' '2020-07-01' '2020-07-09' '2020-07-15'
 '2020-07-21' '2020-07-30' '2020-08-06' '2020-08-13' '2020-08-20'
 '2020-09-09']
Index(['Date', 'Row', 'Panel', 'Treatment', 'Block', 'PM_severity',
       'DM_severity', 'total_dis', 'geometry', 'centroid'],
      dtype='object')


In [16]:
# Match scout data to image data

scout_csv = pd.read_csv('scout_2020_gdf.csv')

# Define filtering and matching parameters

first_row = 1
last_row = 10

match_dates = ['2020-06-25','2020-07-09', '2020-08-13']
match_cols =['row', 'panel', 'Date']


# Filter and join datasets

img_filtered = appended[appended['row']>=first_row]
img_filtered = img_filtered[img_filtered['row']<=last_row]
scout_filtered = scout_csv[scout_csv['Date'].isin(match_dates)]
scout_filtered = scout_filtered.rename(columns={'Row':'row', 'Panel':'panel'})

# Create a 'Date' column in the image dataframe to match the scout date
def match_days(row):
    if row['acquired']=='20200625':
        return '2020-06-25'
    if row['acquired']=='20200710':
        return '2020-07-09'
    if row['acquired']=='20200812':
        return '2020-08-13'

img_filtered['Date'] = img_filtered.apply(lambda row: match_days(row), axis=1)

# Join and drop NANs + unneeded columns
joined = img_filtered.join(scout_filtered.set_index(match_cols), on=match_cols)
cleaned = joined.dropna(axis=0,subset=['Treatment', 'Block', 'DM_severity', 'PM_severity', 'total_dis', 'geometry', 'centroid'])
#cleaned = joined.dropna(axis=0, subset=['geometry','Block', 'total_dis','centroid'])

In [17]:
cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 300 entries, 1 to 158
Data columns (total 16 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   acquired     300 non-null    object 
 1   plot         300 non-null    int64  
 2   blue         300 non-null    float64
 3   green        300 non-null    float64
 4   red          300 non-null    float64
 5   nir          300 non-null    float64
 6   row          300 non-null    int64  
 7   panel        300 non-null    int64  
 8   Date         300 non-null    object 
 9   Treatment    300 non-null    float64
 10  Block        300 non-null    object 
 11  PM_severity  300 non-null    float64
 12  DM_severity  300 non-null    float64
 13  total_dis    300 non-null    float64
 14  geometry     300 non-null    object 
 15  centroid     300 non-null    object 
dtypes: float64(8), int64(3), object(5)
memory usage: 39.8+ KB


In [9]:
# If using a raster with masked pixels

no_nullpix = cleaned.dropna(axis=0, subset=['blue', 'green', 'red', 'nir'])
no_nullpix.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 267 entries, 4 to 158
Data columns (total 16 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   acquired     267 non-null    object 
 1   plot         267 non-null    int64  
 2   blue         267 non-null    float64
 3   green        267 non-null    float64
 4   red          267 non-null    float64
 5   nir          267 non-null    float64
 6   row          267 non-null    int64  
 7   panel        267 non-null    int64  
 8   Date         267 non-null    object 
 9   Treatment    267 non-null    float64
 10  Block        267 non-null    object 
 11  PM_severity  267 non-null    float64
 12  DM_severity  267 non-null    float64
 13  total_dis    267 non-null    float64
 14  geometry     267 non-null    object 
 15  centroid     267 non-null    object 
dtypes: float64(8), int64(3), object(5)
memory usage: 35.5+ KB


In [18]:
# Export dataframe as .csv

output_fp = '/Users/kathleenkanaley/Desktop/grapes_in_space/stats_work/SkySat_2020/analysis_ready/'

#cleaned.to_csv(output_fp+'skysat_2020_nomask.csv', index=False)